1 Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.



In [235]:
import matplotlib.pyplot as plt
import numpy
import pandas

import seaborn
seaborn.set_context('talk')

In [236]:
import plotly
plotly.__version__
# Make sure it's 4.14.3

'4.14.3'

In [237]:
melb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
melb_df[:]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,Wheelers Hill,12 Strada Cr,4,h,1245000.0,S,Barry,26/08/2017,16.7,3150.0,...,2.0,2.0,652.0,NaN,1981.0,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392.0
13576,Williamstown,77 Merrett Dr,3,h,1031000.0,SP,Williams,26/08/2017,6.8,3016.0,...,2.0,2.0,333.0,133.0,1995.0,NaN,-37.85927,144.87904,Western Metropolitan,6380.0
13577,Williamstown,83 Power St,3,h,1170000.0,S,Raine,26/08/2017,6.8,3016.0,...,2.0,4.0,436.0,NaN,1997.0,NaN,-37.85274,144.88738,Western Metropolitan,6380.0
13578,Williamstown,96 Verdon St,4,h,2500000.0,PI,Sweeney,26/08/2017,6.8,3016.0,...,1.0,5.0,866.0,157.0,1920.0,NaN,-37.85908,144.89299,Western Metropolitan,6380.0


In [238]:
melb_df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [239]:
def remove_outlier(df_in, col_name, iqr_factor):
    """
  IQR outliers removal (https://machinelearningmastery.com/how-to-use-statistics-to-identify-outliers-in-data/)
  accept a dataframe, remove outliers, return cleaned data in a new dataframe
  see http://www.itl.nist.gov/div898/handbook/prc/section1/prc16.htm
  """
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3 - q1  # interquartile range
    fence_low = q1 - iqr_factor * iqr
    fence_high = q3 + iqr_factor * iqr
    df_inliers = df_in.loc[
        (df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)
    ]
    df_outliers = df_in.loc[
        (df_in[col_name] < fence_low) | (df_in[col_name] > fence_high)
    ]
    return df_inliers, df_outliers

In [240]:
iqr_factor = 2.5
df_main_inliers, df_main_outliers = remove_outlier(melb_df, "Price", iqr_factor)

display(len(df_main_inliers) / len(melb_df))

0.9846833578792342

2.Investigar la distribución de las variables del conjunto de datos y seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad.

Justificar cada columna no seleccionada.
Para las variables categóricas seleccionadas, agrupe o combine las categorías poco frecuentes para asegurar que todos los grupos tengan un número mínimo de registros.

In [241]:
cols = df_main_inliers[melb_df == 0].count(axis=0)
cols[cols > 0]
#deteccion de valores nulos en las columnas

Distance           6
Bedroom2          16
Bathroom          34
Car             1021
Landsize        1937
BuildingArea      16
dtype: int64

In [242]:
df_main_inliers_copy=df_main_inliers.copy()
df_main_inliers_copy=df_main_inliers_copy.drop(['Bedroom2'], axis=1)
#se retira la variable bedroom2
df_main_inliers_copy.sample(10,random_state=123)


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
8930,Reservoir,16 Muriel Ct,3,h,825000.0,S,RW,1/07/2017,12.0,3073.0,2.0,5.0,537.0,NaN,NaN,Darebin,-37.70047,145.02881,Northern Metropolitan,21650.0
3350,Ivanhoe,9/88 Marshall St,2,u,480000.0,VB,Miles,3/09/2016,7.9,3079.0,1.0,1.0,0.0,NaN,NaN,Banyule,-37.77000,145.04720,Eastern Metropolitan,5549.0
9907,Hampton,1 Grout St,3,h,1325000.0,VB,RT,24/06/2017,13.8,3188.0,1.0,2.0,400.0,NaN,NaN,Unavailable,-37.93434,145.00816,Southern Metropolitan,5454.0
4913,Preston,16 Opal St,2,h,600000.0,S,Nelson,10/09/2016,8.8,3072.0,1.0,2.0,220.0,106.0,2005.0,Darebin,-37.74420,145.01680,Northern Metropolitan,14577.0
2670,Footscray,8 Jerrold St,2,h,752000.0,SP,Jas,7/05/2016,6.4,3011.0,1.0,0.0,215.0,98.0,1910.0,Maribyrnong,-37.79220,144.89100,Western Metropolitan,7570.0
2107,Collingwood,182 Keele St,2,h,670000.0,S,Jellis,18/03/2017,1.6,3066.0,1.0,0.0,118.0,NaN,NaN,Yarra,-37.79760,144.99300,Northern Metropolitan,4553.0
2361,Essendon,65 Napier St,3,h,1467000.0,S,Greg,7/11/2016,8.0,3040.0,2.0,2.0,188.0,171.0,1900.0,Moonee Valley,-37.75320,144.91780,Western Metropolitan,9264.0
2109,Collingwood,5/22 Stanley St,2,u,515000.0,S,Peter,18/03/2017,1.6,3066.0,1.0,1.0,816.0,NaN,NaN,Yarra,-37.80200,144.98480,Northern Metropolitan,4553.0
1971,Coburg,9 Vincent St,4,h,1155000.0,S,Nelson,18/03/2017,7.8,3058.0,1.0,2.0,370.0,NaN,NaN,Moreland,-37.74310,144.95600,Northern Metropolitan,11204.0
2640,Flemington,12/132 Princes St,2,u,556000.0,S,Woodards,16/07/2016,4.4,3031.0,2.0,1.0,0.0,79.0,2009.0,Moonee Valley,-37.78400,144.93560,Northern Metropolitan,3593.0


In [243]:
df_filtrado = df_main_inliers[['Rooms','Type','Price','Suburb','Date','Distance','Postcode','Lattitude',
            'Longtitude','Bathroom','Car','Landsize','BuildingArea','YearBuilt','Regionname','Propertycount' ]]
df_filtrado
#saco sellerG al no importar quien vendio y method al entender que no importa como fue que se vendio.
#agrego suburb, regionname, propertycount
#creo que hay muchas variables de ubicacion. Habria q ver cual sirve y las otras descartar.

,Rooms,Type,Price,Suburb,Date,Distance,Postcode,Lattitude,Longtitude,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Regionname,Propertycount
0,2,h,1480000.0,Abbotsford,3/12/2016,2.5,3067.0,-37.79960,144.99840,1.0,1.0,202.0,NaN,NaN,Northern Metropolitan,4019.0
1,2,h,1035000.0,Abbotsford,4/02/2016,2.5,3067.0,-37.80790,144.99340,1.0,0.0,156.0,79.0,1900.0,Northern Metropolitan,4019.0
2,3,h,1465000.0,Abbotsford,4/03/2017,2.5,3067.0,-37.80930,144.99440,2.0,0.0,134.0,150.0,1900.0,Northern Metropolitan,4019.0
3,3,h,850000.0,Abbotsford,4/03/2017,2.5,3067.0,-37.79690,144.99690,2.0,1.0,94.0,NaN,NaN,Northern Metropolitan,4019.0
4,4,h,1600000.0,Abbotsford,4/06/2016,2.5,3067.0,-37.80720,144.99410,1.0,2.0,120.0,142.0,2014.0,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,4,h,1245000.0,Wheelers Hill,26/08/2017,16.7,3150.0,-37.90562,145.16761,2.0,2.0,652.0,NaN,1981.0,South-Eastern Metropolitan,7392.0
13576,3,h,1031000.0,Williamstown,26/08/2017,6.8,3016.0,-37.85927,144.87904,2.0,2.0,333.0,133.0,1995.0,Western Metropolitan,6380.0
13577,3,h,1170000.0,Williamstown,26/08/2017,6.8,3016.0,-37.85274,144.88738,2.0,4.0,436.0,NaN,1997.0,Western Metropolitan,6380.0
13578,4,h,2500000.0,Williamstown,26/08/2017,6.8,3016.0,-37.85908,144.89299,1.0,5.0,866.0,157.0,1920.0,Western Metropolitan,6380.0


In [244]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13372 entries, 0 to 13579
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rooms          13372 non-null  int64  
 1   Type           13372 non-null  object 
 2   Price          13372 non-null  float64
 3   Suburb         13372 non-null  object 
 4   Date           13372 non-null  object 
 5   Distance       13372 non-null  float64
 6   Postcode       13372 non-null  float64
 7   Lattitude      13372 non-null  float64
 8   Longtitude     13372 non-null  float64
 9   Bathroom       13372 non-null  float64
 10  Car            13310 non-null  float64
 11  Landsize       13372 non-null  float64
 12  BuildingArea   6998 non-null   float64
 13  YearBuilt      8064 non-null   float64
 14  Regionname     13372 non-null  object 
 15  Propertycount  13372 non-null  float64
dtypes: float64(11), int64(1), object(4)
memory usage: 2.4+ MB


In [245]:
df_filtrado.Type.value_counts()


h    9244
u    3016
t    1112
Name: Type, dtype: int64

In [246]:
df_filtrado.Suburb.value_counts().tail(59)

Clarinda              4
Campbellfield         4
Frankston North       4
Coolaroo              4
Chelsea Heights       4
Bonbeach              4
Aspendale Gardens     4
Kurunjang             4
Wyndham Vale          4
Hampton Park          4
Croydon Hills         4
Knoxfield             4
Kilsyth               4
The Basin             4
Ardeer                3
Springvale South      3
Burnside Heights      3
Docklands             3
Pakenham              3
Notting Hill          3
Seabrook              3
St Helena             3
Cranbourne            3
Chirnside Park        2
Williams Landing      2
Mount Evelyn          2
North Warrandyte      2
Keilor Lodge          2
Waterways             2
Burnside              2
Hurstbridge           2
Cranbourne North      2
Skye                  2
Silvan                2
Deepdene              2
Beaconsfield          2
Diggers Rest          2
Beaconsfield Upper    1
Upwey                 1
Plumpton              1
Bacchus Marsh         1
Bullengarook    

In [247]:
#agrupo todos las categorias que tengan una frecuencia menor a 5
suburb_replace=df_filtrado.Suburb.value_counts().tail(59).index.tolist()
df_filtrado.replace(to_replace=suburb_replace, value='others', inplace=True)


/home/ariel/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4379: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [248]:
df_filtrado.Suburb.value_counts()

Reservoir         359
Richmond          258
Bentleigh East    249
Preston           239
Brunswick         222
                 ... 
Derrimut            5
Kings Park          5
Truganina           5
Lower Plenty        5
Princes Hill        5
Name: Suburb, Length: 256, dtype: int64

In [249]:

df_filtrado[['Suburb','Price']].groupby( 'Suburb').describe()

Price                                                    \
              count          mean            std       min        25%   
Suburb                                                                  
Abbotsford     56.0  1.060366e+06  361279.965616  300000.0   866250.0   
Aberfeldie     43.0  1.216465e+06  589398.860375  280000.0   743000.0   
Airport West   67.0  7.566567e+05  170008.849643  440000.0   666000.0   
Albanvale       6.0  5.370833e+05   78723.831631  415000.0   511125.0   
Albert Park    66.0  1.847023e+06  584128.064408  442500.0  1466250.0   
...             ...           ...            ...       ...        ...   
Windsor        60.0  9.721167e+05  556513.064755  275000.0   546250.0   
Wollert         9.0  5.018111e+05  113115.852607  390000.0   401000.0   
Yallambie      24.0  8.396875e+05  213571.839685  602500.0   720750.0   
Yarraville    164.0  9.886155e+05  343162.909944  330000.0   777500.0   
others        133.0  7.253134e+05  280971.594444  285000.0   537000.0   

                                               
                    50%        75%        max  
Suburb                                         
Abbotsford    1093500.0  1311250.0  1876000.0  
Aberfeldie    1300000.0  1648000.0  2905000.0  
Airport West   755000.0   858000.0  1250000.0  
Albanvale      540750.0   562500.0   655000.0  
Albert Park   1830000.0  2137500.0  3010000.0  
...                 ...        ...        ...  
Windsor        817500.0  1350000.0  2368000.0  
Wollert        525000.0   530000.0   730000.0  
Yallambie      773000.0   920250.0  1670000.0  
Yarraville     953000.0  1190000.0  2450000.0  
others         650000.0   840000.0  1830000.0  

[256 rows x 8 columns]

In [250]:
df_filtrado.Suburb.value_counts().tail(50)

Caroline Springs      11
Oakleigh East         11
Tarneit               10
Travancore            10
Ringwood North        10
Watsonia North        10
Rowville              10
Wollert                9
Springvale             9
Edithvale              9
Mernda                 9
Brooklyn               9
Keilor                 9
Carrum                 9
Ripponlea              9
Caulfield East         9
McKinnon               8
Gisborne               8
Melton West            8
Briar Hill             8
Scoresby               8
Bayswater North        8
Doveton                7
Huntingdale            7
Croydon North          7
Burnley                7
Aspendale              7
Cairnlea               7
Taylors Hill           7
Narre Warren           7
Delahey                6
Dallas                 6
Langwarrin             6
Strathmore Heights     6
Albanvale              6
Diamond Creek          6
Gardenvale             6
Altona Meadows         6
Seaholme               6
Doreen                 6


In [251]:
df_filtrado.Postcode.value_counts()

3073.0    359
3020.0    306
3121.0    290
3040.0    289
3046.0    284
         ... 
3775.0      1
3777.0      1
3809.0      1
3808.0      1
3793.0      1
Name: Postcode, Length: 198, dtype: int64

In [252]:
#hago el mismo analisis para postcode
df_filtrado.Postcode.value_counts().tail(35)

3803.0    5
3802.0    5
3093.0    5
3137.0    4
3180.0    4
3976.0    4
3200.0    4
3024.0    4
3061.0    4
3154.0    4
3008.0    3
3022.0    3
3810.0    3
3796.0    2
3335.0    2
3807.0    2
3099.0    2
3027.0    2
3427.0    2
3116.0    2
3795.0    2
3782.0    1
3757.0    1
3115.0    1
3340.0    1
3158.0    1
3431.0    1
3765.0    1
3756.0    1
3438.0    1
3775.0    1
3777.0    1
3809.0    1
3808.0    1
3793.0    1
Name: Postcode, dtype: int64

In [253]:
#agrupo todos las categorias que tengan una frecuencia menor o igual a 5
postcode_replace=df_filtrado.Postcode.value_counts().tail(35).index.tolist()
df_filtrado.replace(to_replace=postcode_replace, value=3803,inplace=True)

#remplazo por la moda de los valores mas frecuentes

/home/ariel/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4379: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [254]:
df_filtrado.Postcode.value_counts()

3073.0    359
3020.0    306
3121.0    290
3040.0    289
3046.0    284
         ... 
3177.0      7
3805.0      7
3113.0      7
3089.0      6
3910.0      6
Name: Postcode, Length: 164, dtype: int64

In [255]:
df_filtrado.Regionname.value_counts()
#mismo analisis para esta variable

Southern Metropolitan         4510
Northern Metropolitan         3877
Western Metropolitan          2944
Eastern Metropolitan          1466
South-Eastern Metropolitan     449
Eastern Victoria                53
Northern Victoria               41
Western Victoria                32
Name: Regionname, dtype: int64

In [256]:
#agrupo todos las categorias que contienen el valor victoria
regionname_replace=df_filtrado.Regionname.value_counts().tail(3).index.tolist()
df_filtrado.replace(to_replace=regionname_replace, value='Victoria',inplace=True)
regionname_replace
#remplazo por la moda de el valor Victoria que es donde se encuentra, sin discriminar por posicion geografica

['Eastern Victoria', 'Northern Victoria', 'Western Victoria']

In [257]:
df_filtrado.Regionname.value_counts()

Southern Metropolitan         4510
Northern Metropolitan         3877
Western Metropolitan          2944
Eastern Metropolitan          1466
South-Eastern Metropolitan     449
Victoria                       126
Name: Regionname, dtype: int64

In [258]:
df_filtrado['date'] = pandas.to_datetime(df_filtrado.Date)
#se convierte a formato tiempo para poder graficar

<ipython-input-258-b184a7e5132f>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



3. Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico. 
  1. Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana o el mínimo.
  1. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.
  2. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.

In [259]:
# data source:
# https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]
airbnb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
    usecols=interesting_cols)

/home/ariel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning:

Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.



In [260]:
airbnb_df[:3]

,description,neighborhood_overview,street,neighborhood,city,suburb,state,zipcode,latitude,longitude,price,weekly_price,monthly_price
0,"House: Clean, New, Modern, Quite, Safe. 10Km f...",Very safe! Family oriented. Older age group.,"Bulleen, VIC, Australia",Balwyn North,Manningham,Bulleen,VIC,3105,-37.772684,145.092133,60,NaN,NaN
1,A large air conditioned room with queen spring...,This hip area is a crossroads between two grea...,"Brunswick East, VIC, Australia",Brunswick,Moreland,Brunswick East,VIC,3057,-37.766505,144.980736,35,200.0,803.0
2,RIGHT IN THE HEART OF ST KILDA! It doesn't get...,A stay at our apartment means you can enjoy so...,"St Kilda, VIC, Australia",St Kilda,Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,159,1253.0,4452.0


In [261]:
airbnb_df.zipcode.value_counts()

3000.0    2491
3000       876
3006.0     860
3182.0     579
3182       556
          ... 
9004.0       1
3001.0       1
8938.0       1
30122        1
3036         1
Name: zipcode, Length: 463, dtype: int64

In [262]:
airbnb_df['zipcode'] = pandas.to_numeric(airbnb_df.zipcode, errors='coerce')

In [263]:
airbnb_df.zipcode.value_counts()

3000.0    3367
3006.0    1268
3182.0    1135
3141.0     881
3004.0     728
          ... 
2010.0       1
3096.0       1
9004.0       1
3785.0       1
7266.0       1
Name: zipcode, Length: 247, dtype: int64

In [264]:
intersection = numpy.intersect1d(
    airbnb_df.zipcode.values, df_filtrado.Postcode.values, assume_unique=False)
print("Airbnb unique zipcodes", len(airbnb_df.zipcode.unique()))
print("Sales unique zipcodes", len(df_filtrado.Postcode.unique()))
print("Common zipcodes", len(intersection))
#hay un mayor nivel de interseccion debido a que modifique los valores de postcode del melb_df

Airbnb unique zipcodes 248
Sales unique zipcodes 164
Common zipcodes 163


In [265]:
print('Records in Sales df with corresponding zipcode form Airbnb df',
      df_filtrado.Postcode.isin(intersection).sum() / len(df_filtrado))
print('Records in Airbnb df with corresponding zipcode form Sales df',
      airbnb_df.zipcode.isin(intersection).sum() / len(airbnb_df))

Records in Sales df with corresponding zipcode form Airbnb df 0.999326951839665
Records in Airbnb df with corresponding zipcode form Sales df 0.8811967678532431


In [266]:
import plotly.express as px
data = df_filtrado.sample(300)
fig = px.scatter_geo(
    data, lat=data.Lattitude, lon=data.Longtitude, color=data.YearBuilt
    )
fig.update_geos(fitbounds="locations")
fig.show()

In [267]:
import plotly.express as px
color_col = 'city'
data = airbnb_df[~airbnb_df[color_col].isna()].sample(300)
fig = px.scatter_geo(
    data, lat=data.latitude, lon=data.longitude, color=data[color_col]
    )
fig.update_geos(fitbounds="locations")
fig.show()

In [268]:
relevant_cols = ['price', 'weekly_price', 'monthly_price', 'zipcode']
## Option 2
# Pass as argument name the new name of the column, and as value a tuple where
# the first value is the original column and the second value is the operation.
airbnb_price_by_zipcode = airbnb_df[relevant_cols].groupby('zipcode')\
   .agg(airbnb_record_count=('price', 'count'),
        airbnb_price_mean=('price', 'mean'),
        airbnb_weekly_price_mean=('weekly_price', 'mean'),
        airbnb_monthly_price_mean=('monthly_price', 'mean'))\
   .reset_index()


In [270]:
airbnb_price_by_zipcode[:10]

,zipcode,airbnb_record_count,airbnb_price_mean,airbnb_weekly_price_mean,airbnb_monthly_price_mean
0,2010.0,1,40.000000,NaN,NaN
1,2134.0,1,50.000000,NaN,NaN
2,2582.0,1,104.000000,NaN,NaN
3,3000.0,3367,150.504307,918.738956,3407.204651
4,3001.0,2,132.500000,NaN,NaN
5,3002.0,197,200.979695,956.379310,3712.714286
6,3003.0,267,130.205993,760.200000,2629.500000
7,3004.0,728,158.271978,1061.171429,3759.925926
8,3006.0,1268,188.851735,1239.827273,4358.556701
9,3008.0,616,177.011364,1025.275862,3222.833333


In [271]:
merged_sales_df = df_filtrado.merge(
    airbnb_price_by_zipcode, how='left',
    left_on='Postcode', right_on='zipcode'
)
merged_sales_df.sample(5)

,Rooms,Type,Price,Suburb,Date,Distance,Postcode,Lattitude,Longtitude,Bathroom,...,BuildingArea,YearBuilt,Regionname,Propertycount,date,zipcode,airbnb_record_count,airbnb_price_mean,airbnb_weekly_price_mean,airbnb_monthly_price_mean
10393,2,u,709500.0,Yarraville,27/05/2017,6.3,3013.0,-37.81133,144.89553,1.0,...,80.0,1970.0,Western Metropolitan,6543.0,2017-05-27,3013.0,82.0,135.609756,923.571429,2966.090909
10985,2,h,1310000.0,Prahran,12/08/2017,4.6,3181.0,-37.85083,144.99092,2.0,...,NaN,NaN,Southern Metropolitan,7717.0,2017-12-08,3181.0,460.0,148.169565,868.148148,3304.382353
3001,2,u,896000.0,Hampton,19/11/2016,13.7,3188.0,-37.94480,145.01860,2.0,...,NaN,NaN,Southern Metropolitan,5454.0,2016-11-19,3188.0,71.0,245.112676,1273.600000,4694.555556
7152,5,h,1030000.0,West Footscray,30/07/2016,8.2,3012.0,-37.79080,144.87110,2.0,...,NaN,NaN,Western Metropolitan,5058.0,2016-07-30,3012.0,76.0,112.631579,518.000000,2335.000000
8541,3,t,1320000.0,Toorak,29/04/2017,4.6,3142.0,-37.84740,145.01250,2.0,...,131.0,1970.0,Southern Metropolitan,7217.0,2017-04-29,3142.0,118.0,244.500000,1168.823529,4471.833333
